In [1]:
%pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [6]:
import scipy.io as sio
import numpy as np

# Load a .mat file
mat_data = sio.loadmat('Preprocessed-Dataset-with-Integrated-Annotations/01/MPDDF_preprocessed_01_ECG.mat')

# View what's in the file
print(mat_data.keys())

# Access a specific variable
data = mat_data['ecg_data']

# Work with the data
print(data.shape)
print(type(data))
print(data[:5])

dict_keys(['__header__', '__version__', '__globals__', 'ecg_data', 'annotation'])
(7202, 1, 1024)
<class 'numpy.ndarray'>
[[[ 3.81475547e-08 -1.48775463e-06 -5.76028077e-06 ...  4.31067369e-06
   -2.93736172e-06 -6.98100252e-06]]

 [[-4.76844434e-06 -3.01365682e-06 -6.21805142e-06 ...  6.70366979e-04
    7.09735256e-04  7.57419699e-04]]

 [[ 8.17845426e-04  8.83764401e-04  9.45944915e-04 ... -1.00328069e-05
   -1.37712673e-05 -1.36949722e-05]]

 [[-9.26985580e-06 -8.58319982e-06 -1.44579232e-05 ...  4.20004578e-05
    3.41420615e-05  3.36079957e-05]]

 [[ 4.08560311e-05  4.72648203e-05  4.92484932e-05 ...  5.61150530e-05
    4.46707866e-05  3.85671778e-05]]]


In [1]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt

# Path to your ECG file
mat_path = "Preprocessed-Dataset-with-Integrated-Annotations/31/MPDDF_preprocessed_31_ECG.mat"
m_path = "Preprocessed-Dataset-with-Integrated-Annotations"
# Load .mat file
label1=0
label2=0
label3=0
label4=0
label5=0
label6=0
label7=0
label8=0
label9=0
for i in range(1,51):
    mat_path = f"{m_path}/{i:02d}/MPDDF_preprocessed_{i:02d}_ECG.mat"
    data = sio.loadmat(mat_path)

    # See available keys
    # print("Keys in .mat file:")
    # print(data.keys())

    # Extract annotation
    annotation = data['annotation']   # usually shape (1, N)

    # print("\nRaw annotation shape:", annotation.shape)

    # Flatten to 1D
    annotation = annotation.squeeze()
    # print("After squeeze:", annotation.shape)

    # Unique labels and counts
    unique_labels, counts = np.unique(annotation, return_counts=True)

    
    d = []
    for label, count in zip(unique_labels, counts):
        if label == 1:
            label1 += count
        elif label == 2:
            label2 += count
        elif label == 3:
            label3 += count
        elif label == 4:
            label4 += count
        elif label == 5:
            label5 += count
        elif label == 6:
            label6 += count
        elif label == 7:
            label7 += count
        elif label == 8:
            label8 += count
        elif label == 9:
            label9 += count
        d.append((label, count))
        # print(f"Label {label}: {count} samples")
    print(f"File {i:02d} - Label distribution: {d}")

arr = [label1, label2, label3, label4, label5, label6, label7, label8, label9]

print(arr)

# Plot label over time
# plt.figure(figsize=(15,4))
# plt.plot(annotation)
# plt.title("Annotation over time (1 label per second)")
# plt.xlabel("Time (seconds)")
# plt.ylabel("Label")
# plt.show()


File 01 - Label distribution: [(np.int64(0), np.int64(6556)), (np.int64(1), np.int64(646))]
File 02 - Label distribution: [(np.int64(0), np.int64(4849)), (np.int64(1), np.int64(1931)), (np.int64(2), np.int64(422))]
File 03 - Label distribution: [(np.int64(0), np.int64(6386)), (np.int64(1), np.int64(818))]
File 04 - Label distribution: [(np.int64(0), np.int64(2246)), (np.int64(1), np.int64(2388)), (np.int64(2), np.int64(2403)), (np.int64(3), np.int64(157))]
File 05 - Label distribution: [(np.int64(0), np.int64(1374)), (np.int64(1), np.int64(1884)), (np.int64(9), np.int64(3943))]
File 06 - Label distribution: [(np.int64(0), np.int64(5678)), (np.int64(1), np.int64(1464)), (np.int64(2), np.int64(60))]
File 07 - Label distribution: [(np.int64(0), np.int64(6819)), (np.int64(1), np.int64(213)), (np.int64(2), np.int64(169))]
File 08 - Label distribution: [(np.int64(0), np.int64(7175)), (np.int64(1), np.int64(21))]
File 09 - Label distribution: [(np.int64(0), np.int64(1892)), (np.int64(1), np.i

In [2]:
import os
import scipy.io as sio
import numpy as np
from tqdm import tqdm

# -------- SETTINGS --------
data_root = "Preprocessed-Dataset-with-Integrated-Annotations"
window_size = 10      # 10 seconds
sampling_rate = 1024  # ECG sampling rate
samples_per_window = window_size * sampling_rate

X_all = []
y_all = []
subj_all = []

# Loop over all subject folders
subject_ids = sorted(os.listdir(data_root))

for subj in tqdm(subject_ids):
    subj_path = os.path.join(data_root, subj)
    
    # ECG file name pattern
    ecg_file = [f for f in os.listdir(subj_path) if "ECG" in f][0]
    mat = sio.loadmat(os.path.join(subj_path, ecg_file))
    
    ecg = mat['ecg_data']         # shape (N_sec, 1, 1024)
    annotation = mat['annotation'].squeeze()  # shape (N_sec,)
    
    # Remove artifact seconds (8, 9)
    valid_mask = np.isin(annotation, [0,1,2,3])
    ecg = ecg[valid_mask]
    annotation = annotation[valid_mask]
    
    # Convert to binary labels
    annotation = np.where(annotation == 0, 0, 1)
    
    # Remove channel dimension
    ecg = ecg.squeeze(1)  # shape (N_sec, 1024)
    
    # Number of full 10-sec windows
    total_seconds = ecg.shape[0]
    num_windows = total_seconds // window_size
    
    for i in range(num_windows):
        start = i * window_size
        end = start + window_size
        
        ecg_block = ecg[start:end]  # shape (10, 1024)
        label_block = annotation[start:end]
        
        # Majority label
        majority_label = np.bincount(label_block).argmax()
        
        # Reshape ECG to (10240,)
        ecg_block = ecg_block.reshape(-1)
        
        X_all.append(ecg_block)
        y_all.append(majority_label)
        subj_all.append(int(subj))  # keep subject ID for LOSO

# Convert to numpy arrays
X_all = np.array(X_all, dtype=np.float32)
y_all = np.array(y_all)
subj_all = np.array(subj_all)

print("Final dataset shape:", X_all.shape)
print("Labels shape:", y_all.shape)

# Show distribution
unique, counts = np.unique(y_all, return_counts=True)
print("Binary label distribution:")
for u, c in zip(unique, counts):
    print(f"Label {u}: {c}")


100%|██████████| 50/50 [00:08<00:00,  6.08it/s]


Final dataset shape: (36388, 10240)
Labels shape: (36388,)
Binary label distribution:
Label 0: 27899
Label 1: 8489


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ECG_CNN_10s(nn.Module):
    def __init__(self):
        super(ECG_CNN_10s, self).__init__()
        
        self.conv1 = nn.Conv1d(1, 32, kernel_size=7, stride=1, padding=3)
        self.bn1 = nn.BatchNorm1d(32)
        self.pool1 = nn.MaxPool1d(4)
        
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.pool2 = nn.MaxPool1d(4)
        
        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.pool3 = nn.MaxPool1d(4)
        
        self.conv4 = nn.Conv1d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm1d(256)
        
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        
        self.fc1 = nn.Linear(256, 64)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(64, 2)  # binary classification

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
        
        x = F.relu(self.bn4(self.conv4(x)))
        
        x = self.global_pool(x)
        x = x.squeeze(-1)
        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x


In [4]:
import torch

X_tensor = torch.tensor(X_all).unsqueeze(1)  # (N, 1, 10240)
y_tensor = torch.tensor(y_all).long()


In [5]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_all),
    y=y_all
)

weights = torch.tensor(class_weights, dtype=torch.float32)

criterion = nn.CrossEntropyLoss(weight=weights)


In [7]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = ECG_CNN_10s().to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_all,
    y_all,
    test_size=0.2,
    stratify=y_all,
    random_state=42
)


In [12]:
import torch
from torch.utils.data import Dataset, DataLoader

class ECGDataset(Dataset):
    def __init__(self, X, y):
        X = torch.tensor(X, dtype=torch.float32)
        
        # Per-sample normalization
        mean = X.mean(dim=1, keepdim=True)
        std = X.std(dim=1, keepdim=True) + 1e-8
        X = (X - mean) / std
        
        self.X = X.unsqueeze(1)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


train_dataset = ECGDataset(X_train, y_train)
test_dataset  = ECGDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [10]:
import torch.nn as nn
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ECG_CNN_10s().to(device)

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [11]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import numpy as np

num_epochs = 30
best_auc = 0
patience = 5
patience_counter = 0

for epoch in range(num_epochs):
    
    # ---- TRAIN ----
    model.train()
    train_loss = 0
    
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    # ---- EVAL ----
    model.eval()
    all_preds = []
    all_probs = []
    all_labels = []
    
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            
            outputs = model(X_batch)
            probs = torch.softmax(outputs, dim=1)[:, 1]
            preds = torch.argmax(outputs, dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())
    
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs)
    
    print(f"Epoch {epoch+1}")
    print(f"Train Loss: {train_loss/len(train_loader):.4f}")
    print(f"Test Acc: {acc:.4f} | F1: {f1:.4f} | AUC: {auc:.4f}")
    
    # ---- Early Stopping ----
    if auc > best_auc:
        best_auc = auc
        torch.save(model.state_dict(), "best_ecg_model.pth")
        patience_counter = 0
    else:
        patience_counter += 1
    
    if patience_counter >= patience:
        print("Early stopping triggered.")
        break


Epoch 1
Train Loss: 0.5978
Test Acc: 0.7667 | F1: 0.0000 | AUC: 0.5191
Epoch 2
Train Loss: 0.5283
Test Acc: 0.7667 | F1: 0.0000 | AUC: 0.5525
Epoch 3
Train Loss: 0.5081
Test Acc: 0.7667 | F1: 0.0000 | AUC: 0.5930
Epoch 4
Train Loss: 0.4946
Test Acc: 0.2333 | F1: 0.3783 | AUC: 0.5013
Epoch 5
Train Loss: 0.4874
Test Acc: 0.2333 | F1: 0.3782 | AUC: 0.5381
Epoch 6
Train Loss: 0.4803
Test Acc: 0.7667 | F1: 0.0000 | AUC: 0.5456
Epoch 7
Train Loss: 0.4752
Test Acc: 0.2332 | F1: 0.3780 | AUC: 0.4996
Epoch 8
Train Loss: 0.4649
Test Acc: 0.2332 | F1: 0.3780 | AUC: 0.4921
Early stopping triggered.


In [14]:
import torch.nn as nn
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ECG_CNN_10s().to(device)

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)


In [15]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import numpy as np

num_epochs = 30
best_auc = 0
patience = 5
patience_counter = 0

for epoch in range(num_epochs):
    
    # ---- TRAIN ----
    model.train()
    train_loss = 0
    
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    # ---- EVAL ----
    model.eval()
    all_preds = []
    all_probs = []
    all_labels = []
    
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            
            outputs = model(X_batch)
            probs = torch.softmax(outputs, dim=1)[:, 1]
            preds = torch.argmax(outputs, dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())
    
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs)
    
    print(f"Epoch {epoch+1}")
    print(f"Train Loss: {train_loss/len(train_loader):.4f}")
    print(f"Test Acc: {acc:.4f} | F1: {f1:.4f} | AUC: {auc:.4f}")
    
    # ---- Early Stopping ----
    if auc > best_auc:
        best_auc = auc
        torch.save(model.state_dict(), "best_ecg_model2.pth")
        patience_counter = 0
    else:
        patience_counter += 1
    
    if patience_counter >= patience:
        print("Early stopping triggered.")
        break


Epoch 1
Train Loss: 0.5730
Test Acc: 0.6708 | F1: 0.5447 | AUC: 0.8276
Epoch 2
Train Loss: 0.5183
Test Acc: 0.7770 | F1: 0.5996 | AUC: 0.8416
Epoch 3
Train Loss: 0.4996
Test Acc: 0.8306 | F1: 0.6179 | AUC: 0.8371
Epoch 4
Train Loss: 0.4914
Test Acc: 0.7872 | F1: 0.6136 | AUC: 0.8587
Epoch 5
Train Loss: 0.4785
Test Acc: 0.8190 | F1: 0.6188 | AUC: 0.8608
Epoch 6
Train Loss: 0.4683
Test Acc: 0.8097 | F1: 0.6202 | AUC: 0.8538
Epoch 7
Train Loss: 0.4672
Test Acc: 0.6832 | F1: 0.5656 | AUC: 0.8594
Epoch 8
Train Loss: 0.4608
Test Acc: 0.7579 | F1: 0.6060 | AUC: 0.8683
Epoch 9
Train Loss: 0.4583
Test Acc: 0.7458 | F1: 0.5982 | AUC: 0.8631
Epoch 10
Train Loss: 0.4519
Test Acc: 0.7270 | F1: 0.5988 | AUC: 0.8770
Epoch 11
Train Loss: 0.4504
Test Acc: 0.8435 | F1: 0.6374 | AUC: 0.8828
Epoch 12
Train Loss: 0.4462
Test Acc: 0.8182 | F1: 0.6356 | AUC: 0.8761
Epoch 13
Train Loss: 0.4424
Test Acc: 0.8226 | F1: 0.6468 | AUC: 0.8825
Epoch 14
Train Loss: 0.4377
Test Acc: 0.7205 | F1: 0.5958 | AUC: 0.8750
E